In [226]:
import re
import os
import pandas as pd
from termcolor import colored
import nltk
from nltk.stem import PorterStemmer

In [306]:
def clean(text,shrink=False):
    text=text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text=re.sub('[“"”]',' " ',text)
    retain='[^abcdefghijklmnopqrstuvwxyz!#?" ]'
    text=re.sub('[()–-]',' ',text)
    text=re.sub(retain,'',text)
    text=text.replace('?',' ? ')
    text=text.replace('#',' # ')
    text=text.replace('!',' ! ')
    text=text.split()
    Lemmatizer=WordNetLemmatizer()
    Stemmer=PorterStemmer()
    if shrink:    
        text=[Stemmer.stem(Lemmatizer.lemmatize(word, pos='v')) for word in text]
    return ' '.join(text)

In [77]:
directory='datasets/train-task2-TC.labels'
props_=open(directory).read().split('\n')[:-1]
sentences,Articles, labels=[],[],[]
IDS=[]
Spans=[]
for prop in props_:
    prop_=prop.split('\t')
    x,y=int(prop_[2]),int(prop_[3])
    Spans.append((x,y))
    sentences.append([open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[:x],open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[x:y],open('datasets/train-articles/article{}.txt'.format(prop_[0])).read()[y:]])
    Articles.append(open('datasets/train-articles/article{}.txt'.format(prop_[0])).read())
    labels.append(prop_[1])
    IDS.append(prop_[0])
    ids=int(prop_[0])

In [64]:
ind=2
def gen(ind):
    print(labels[ind])
    pre=colored(sentences[ind][0], color='green')
    sen=colored(sentences[ind][1], color='red')
    post=colored(sentences[ind][-1], color='green')
    return(pre+sen+post)

In [157]:
df=pd.DataFrame({'Id':IDS, 'Span':Spans, 'Label':labels})
set(labels)

{'Appeal_to_Authority',
 'Appeal_to_fear-prejudice',
 'Bandwagon,Reductio_ad_hitlerum',
 'Black-and-White_Fallacy',
 'Causal_Oversimplification',
 'Doubt',
 'Exaggeration,Minimisation',
 'Flag-Waving',
 'Loaded_Language',
 'Name_Calling,Labeling',
 'Repetition',
 'Slogans',
 'Thought-terminating_Cliches',
 'Whataboutism,Straw_Men,Red_Herring'}

In [98]:
def span_read(id,span):
    return open('datasets/train-articles/article{}.txt'.format(id)).read()[span[0]:span[1]]

'The next transmission could be more pronounced or stronger'

In [231]:
Stitch=""
for doc in os.listdir('datasets/train-articles'):
    Stitch=Stitch+open('datasets/train-articles/'+doc).read()
corp_freq=dict()
Doc_freqs=dict()

In [252]:
def freq_dict(text):
    freq={}
    for word in clean(text).split():
        if word in freq:
            freq[word]=freq[word]+1
        else:
            freq[word]=1
    return freq

In [253]:
corp_freq=freq_dict(Stitch)
for id in IDS :
    txt=open('datasets/train-articles/article{}.txt'.format(id)).read()
    Doc_freqs[id]=freq_dict(txt)

In [301]:
for i in df[df['Label']=='Doubt'].index.tolist()[:100]:
    print(gen(i))

Doubt
Next plague outbreak in Madagascar could be 'stronger': WHO

Geneva - The World Health Organisation chief on Wednesday said a deadly plague epidemic appeared to have been brought under control in Madagascar, but warned the next outbreak would likely be stronger.

"The next transmission could be more pronounced or stronger," WHO Director-General Tedros Adhanom Ghebreyesus told reporters in Geneva, insisting that "the issue is serious."

An outbreak of both bubonic plague, which is spread by infected rats via flea bites, and pneumonic plague, spread person to person, has killed more than 200 people in the Indian Ocean island nation since August.

Madagascar has suffered bubonic plague outbreaks almost every year since 1980, often caused by rats fleeing forest fires.

The disease tends to make a comeback each hot rainy season, from September to April.
On average, between 300 and 600 infections are recorded every year among a population approaching 25 million people, according to a U

In [399]:
def pad(val):
    t_df=df[df['Label']=='Doubt']
    t_id,t_Span=t_df['Id'],t_df['Span']
    padsent_d=[]
    for ids,spans in zip(t_id,t_Span):
        padsent_d.append(span_read(ids,(spans[0]-val,spans[1]+val)))

    t_df=df[df['Label']!='Doubt']
    t_id,t_Span=t_df['Id'],t_df['Span']
    padsent=[]
    for ids,spans in zip(t_id,t_Span):
        padsent.append(span_read(ids,(spans[0]-val,spans[1]+val)))
    return padsent_d,padsent

In [384]:
#x=freq_dict(clean(' '.join(df[df['Label']=='Doubt']['sentences'].tolist())))
x=freq_dict(clean(' '.join(padsent)))
{k: v for k, v in sorted(x.items(), key=lambda item: item[1]) if v > 9}

{'while': 10,
 'cannot': 10,
 'happened': 10,
 'made': 10,
 'get': 10,
 'after': 10,
 'under': 10,
 'simply': 10,
 'again': 10,
 'your': 10,
 'cant': 10,
 'asked': 10,
 'knew': 10,
 'answer': 10,
 'back': 10,
 'done': 10,
 'didnt': 10,
 'without': 10,
 'popes': 10,
 'entire': 11,
 'believe': 11,
 'american': 11,
 'well': 11,
 'think': 11,
 'instead': 11,
 'anyone': 11,
 'some': 11,
 'history': 11,
 'truth': 11,
 'false': 11,
 'case': 11,
 'd': 11,
 'going': 11,
 'himself': 11,
 'russia': 11,
 'ford': 11,
 'law': 12,
 'really': 12,
 'my': 12,
 'here': 12,
 'bishops': 12,
 'fake': 12,
 'harding': 12,
 'country': 13,
 'question': 13,
 'before': 13,
 'far': 13,
 'very': 13,
 'man': 13,
 'state': 13,
 'right': 13,
 'news': 13,
 'could': 13,
 'gillum': 13,
 'those': 14,
 'him': 14,
 'campaign': 14,
 'fact': 14,
 'much': 14,
 'catholic': 14,
 'where': 14,
 'says': 14,
 'may': 15,
 'other': 15,
 'now': 15,
 'god': 15,
 'new': 15,
 'me': 15,
 'time': 15,
 'francis': 15,
 'into': 16,
 'election'

In [513]:
def print_res(val,wts):
    isin,notin=pad(val)
    c,c1,t,t1=0,0,0,0
    box=['?','how','when','where','why','while','what','may','cant']
    for sent in isin:
        t=t+1
        #cond=False
        for check in box:
            if check in clean(sent).split():
                c=c+wts[check]
    for sent in notin:
        t1=t1+1
        #cond=False
        for check in box:
            if check in clean(sent).split():
                c1=c1+wts[check]
    print(c/t,c)
    print(c1/t1,c1)
    print((c*t1)/(c1*t))

In [589]:
wts={
    '?' : 100,
    'how' : 1,
    'when' : 1,
    'where' : 1,
    'why' :1000,
    'while' : 1,
    'what' : 1,
    'may' : 1,
    'cant' : 1,
}
print_res(20,wts)

100.75 49972
12.633577388046994 74197
7.9747799776271275


In [415]:
clean_stitch=clean(Stitch)
len(clean_stitch)/clean_stitch.count(' ')

5.939610274579274